# Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:

* tracking server: yes, local server
* backend store: sqlite database
* artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

mlflow server --backend-store-uri sqlite:///backend.db

In [1]:
import mlflow

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
mlflow.set_experiment('my-experiment-1')

MlflowException: Cannot set a deleted experiment 'my-experiment-1' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.

In [4]:
with mlflow.start_run() as run:
    
    X,y = load_iris(return_X_y=True)
    
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)
    
    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(lr, artifact_path='models')
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")


default artifacts URI: 'mlflow-artifacts:/0/beca8e073ce54cb19c666e989e19d26f/artifacts'


E:\Estudos\Python\MLOps-Zoomcamp\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685271394140, experiment_id='0', last_update_time=1685271394140, lifecycle_stage='active', name='Default', tags={}>]

# Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


In [8]:
client = MlflowClient("http://127.0.0.1:5000")

In [9]:
client.search_registered_models()

[]

In [19]:
run_id = run.info.run_id

In [20]:
mlflow.register_model(model_uri=f"runs:/{run_id}/models", name="iris-classifier")

Successfully registered model 'iris-classifier'.
2023/05/28 12:06:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1685272015508, current_stage='None', description='', last_updated_timestamp=1685272015508, name='iris-classifier', run_id='beca8e073ce54cb19c666e989e19d26f', run_link='', source='mlflow-artifacts:/0/beca8e073ce54cb19c666e989e19d26f/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>